In [18]:
#! nvidia-smi

In [19]:
#conda install pytorch torchvision -c pytorch

In [20]:
import os
def EXIT_NOTEBOOK(): os._exit(00)
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Changes in code cell 4. added line 3.  

In [21]:
import numpy as np
import matplotlib.pyplot as plt
# plt.rcParams['text.usetex'] = False
from tqdm.notebook import tqdm
import random
import torch 
from torch import nn
import torch.nn.functional as func
import seaborn as sns
import matplotlib.ticker

# # ps
# import pysindy as ps

# sns.set_theme()
torch.set_default_dtype(torch.float64)
plt.rcParams['text.usetex'] = True

# Set the vector field

In [22]:
def get_v(x):
    x = x[...,-1]
    return np.stack([3*x*np.exp(-2*x**2),3*x*np.exp(-2*x**2)-5*x],axis=-1)

In [23]:
dim = 1

# Dataset for X

In [24]:
X = np.random.uniform(-3,3,(10000,1))
get_v(X[0]),get_v(X[:2])

(array([-0.45346994,  4.35619136]),
 array([[-0.45346994,  4.35619136],
        [-0.7004384 ,  0.65046198]]))

# Set the NN model and Solver with training process

In [25]:
def relu2(X): return func.relu(X)**2
def tanh(X): return func.tanh(X)
class FCNN(nn.Module):
    def __init__(self,input_dim=2,output_dim=1,num_hidden=2,hidden_dim=10,act=func.tanh,transform=None):
        super().__init__()
         
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers  = nn.ModuleList([nn.Linear(input_dim,hidden_dim)])
        for _ in range(num_hidden-1): self.layers.append(nn.Linear(hidden_dim,hidden_dim))
        self.act     = act
        self.out     = nn.Linear(hidden_dim,output_dim)
        self.transform = transform
    def forward(self,X):
        if self.transform is not None: X = self.transform(X)
        for layer in self.layers: X = self.act(layer(X))
        Y = self.out(X)
        return Y
class Model(nn.Module):
    def __init__(self,dim,model_U,unit_len=int(5e3)):
        super().__init__()
        self.dim      = dim
        self.model_U  = model_U
        self.unit_len = unit_len
        self.mu       = nn.Parameter(torch.tensor([0.]*dim),requires_grad=False) 
        self.sigma    = nn.Parameter(torch.tensor([1.]*dim),requires_grad=False)
        #above two lines should work, but if something doesn't work, check here! Does current self.mu and self.sigma code work if dim>1. Should return a vector since what we're trying to do is calculate mu
        #and sigma separately for each imnputted feature. mu is parameter[0] for each row.
        self.coef_U   = nn.Parameter(torch.tensor(1.),requires_grad=False)
        #self.mu       = nn.Parameter(torch.tensor([0.]*dim).cuda(),requires_grad=False)
        #self.sigma    = nn.Parameter(torch.tensor([1.]*dim).cuda(),requires_grad=False)
        #self.coef_U   = nn.Parameter(torch.tensor(1.).cuda(),requires_grad=False)
    def get_U_harmonic(self,X): return torch.sum(X**2,axis=-1)


    def get_U_dU(self,X): #X is all data aka all positions
        I = int(np.ceil(len(X)/self.unit_len)) #will have to change the evenness of the split eventually. this is dividing up the data
        U,dU = [],[]
        for i in range(I):
            X_sub = X[i*self.unit_len:(i+1)*self.unit_len] #**needs changing for dimensionality. 1bead to 3bead
            if not torch.is_tensor(X_sub): X_sub = torch.tensor(X_sub,requires_grad=True)#.cuda()
            X_    = (X_sub-self.mu)/self.sigma
            U_    = self.coef_U*(self.model_U(X_).view(-1) + self.get_U_harmonic(X_))
            dU_   = torch.autograd.grad(U_,X_sub,torch.ones_like(U_),create_graph=True)[0] # it is X_sub!!!
            U.append(U_)
            dU.append(dU_)
        U = torch.hstack(U) #if U is a matrix instead of a scalar, will torch.hstack(U) stack the list that is in each element in U? Or something else.
        dU = torch.vstack(dU)
        return U,dU
    
    
    def get_U_np(self,X): 
        U,_ = self.get_U_dU(X);
        return U.cpu().data.numpy()
class Solver():
    def __init__(self,model):
        self.model=model
    def train_model(self,data_train,data_test,get_loss,optimizer,
                    n_steps,batch_size,scheduler=None,n_show_loss=100,error_model=None,use_tqdm=True):
        if use_tqdm: step_range = tqdm(range(n_steps))
        else: step_range = range(n_steps)
        loss_step = []
        for i_step in step_range:
            if i_step%n_show_loss==0:
                loss_train,loss_test = get_loss(self.model,data_train)[:-1],\
                                       get_loss(self.model,data_test)[:-1]
                
                def show_num(x): 
                    if abs(x)<100 and abs(x)>.01: return '%0.5f'%x
                    else: return '%0.2e'%x
                item1 = '%2dk'%np.int_(i_step/1000)
                item2 = 'Loss: '+' '.join([show_num(k) for k in loss_train])
                item3 = ' '.join([show_num(k) for k in loss_test])
                item4 = ''
                if error_model is not None: item4 = 'E(QP): %0.4f' % (error_model(self.model))
                # print(', '.join([item1,item2,item3,item4]))
                loss_step = loss_step + [i_step] + [k.cpu().data.numpy() for k in loss_train]\
                                                 + [k.cpu().data.numpy() for k in loss_train]
            data_batch = data_train[random.sample(range(len(data_train)),
                                                  min(batch_size,len(data_train)))]
#             print(i_step,data_batch.shape)
            loss = get_loss(self.model,data_batch)[-1]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if scheduler is not None: scheduler.step()
        # if error_model is not None: 
            # print("Error: %0.5f" % (error_model(self.model)))
        return loss_step

In [26]:
model_U = FCNN(input_dim=dim,output_dim=1,num_hidden=3,hidden_dim=10,act=tanh)#.cuda()
model   = Model(dim,model_U=model_U)#.cuda();
SOL     = Solver(model)

In [27]:
model.get_U_dU(X[:3])

(tensor([ 0.8144, -0.0310,  7.3798], grad_fn=<CatBackward0>),
 tensor([[-1.9407],
         [-0.4946],
         [ 5.3767]], grad_fn=<CatBackward0>))

# Set the loss function and Train the model for differen a_k(x)

Changed code cell 23. added keepdim on getmean of residue

In [28]:
def plot_model(model,cmap='terrain',max_V = 10):
    
    xx     = np.linspace(0,2,1000).reshape(-1,1)
    U_NN   = model.get_U_np(xx)
    U_NN_min = U_NN.min()
    U_NN  = U_NN-U_NN_min

    fig, ax    = plt.subplots(1,1,figsize=(5,3),dpi=200,constrained_layout=True)
    c      = ax.plot(xx[:,0],U_NN,'-',lw=1.5,)

    ax.tick_params(axis="both", labelsize=10)
    plt.show()
#def get_a(X,k=choose_id):
#    if choose_id==1: return 2*torch.exp(-3*X**2)
#    if choose_id==2: return 2/(1+torch.exp(20*(torch.abs(X)-0.75)))
#    if choose_id==3: return 4/(1+torch.exp(20*(torch.abs(X)-0.75)))
def get_loss(model,data):

    X = data
    X = torch.tensor(X,requires_grad=True)#.cuda()
    _,dU = model.get_U_dU(X)

    #CTMC
    param = 1
    c = 0.5*param
    
    tmp1 = torch.detach(3*X*torch.exp(-2*X**2)) #common part of v1 and v2, aka v1
    tmp2 = torch.detach(-2*param/(1+torch.exp(20*(torch.abs(X)-0.75)))) #affinity fucntion
    #tmp2 = torch.detach(-get_a(X))
    tmp3 = torch.detach(-tmp2)
    tmp4 = torch.detach(tmp1-5*X) #v2
    Res = ( dU*(dU+tmp1)+tmp2 ) * (dU*(dU+tmp4)-c) - c*tmp3 #determinant 
    loss = torch.mean( Res**2, dim= (1), keepdim=True ) #norm of determinant -> loss function
    
    return loss#,loss

plot_model(model)
get_loss(model,X[:3])

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1000x600 with 1 Axes>

tensor([[5.7466e+02],
        [3.4947e-02],
        [1.5776e+06]], grad_fn=<MeanBackward1>)

In [29]:
#for choose_id in [1,2,3]:
model_U = FCNN(input_dim=dim,output_dim=1,num_hidden=3,hidden_dim=10,act=tanh)#.cuda()
model   = Model(dim,model_U=model_U)#.cuda();
SOL     = Solver(model)

# print(model.mu,model.sigma,model.coef_U)
    #optimizer = torch.optim.Adam(model.parameters(), lr=torch.tensor(0.001).cuda())
optimizer = torch.optim.Adam(model.parameters(), lr=torch.tensor(0.001))
scheduler = None
_loss_step = SOL.train_model(data_train=X,data_test=X,
                             get_loss=get_loss,optimizer=optimizer,scheduler=scheduler,
                             n_steps=int(5e4+1),batch_size=500,n_show_loss=1000,use_tqdm=True)
torch.cuda.empty_cache()
plot_model(model)
   # torch.save(model.state_dict(), "savee/model_"+str(choose_id))
#torch.save(model.state_dict(),"savee/model_anaconda3")

  0%|          | 0/50001 [00:00<?, ?it/s]

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1000x600 with 1 Axes>

In [30]:
#torch.save(model.state_dict(), "savee/model_99")
#model.state_dict()

In [31]:
fig, ax  = plt.subplots(1,1,figsize=(4,3),dpi=200,constrained_layout=True)
xx       = np.linspace(0,2,1000).reshape(-1,1)
U_NN     = model.get_U_np(xx)
U_NN_min = U_NN.min()
U_NN     = U_NN-U_NN_min
c        = ax.plot(xx[:,0],U_NN,'-',lw=1.5)
ax.legend(fontsize=10)
# ax.set_xlabel('$x$',fontsize=10)
# ax.set_ylabel('$U(x)$',fontsize=10)
ax.set_xlim([0,2])
ax.set_yticks([0,0.2,0.4,0.6,0.8,1,1.2])
ax.set_xticks([0,.5,1.,1.5,2])
ax.yaxis.grid(linestyle='--')
ax.tick_params(axis="both", labelsize=10)
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 800x600 with 1 Axes>

In [32]:
U_NN

array([4.15928166e-02, 4.15645913e-02, 4.15365993e-02, 4.15088428e-02,
       4.14813239e-02, 4.14540448e-02, 4.14270075e-02, 4.14002141e-02,
       4.13736666e-02, 4.13473672e-02, 4.13213177e-02, 4.12955202e-02,
       4.12699767e-02, 4.12446890e-02, 4.12196591e-02, 4.11948889e-02,
       4.11703803e-02, 4.11461351e-02, 4.11221551e-02, 4.10984421e-02,
       4.10749979e-02, 4.10518242e-02, 4.10289229e-02, 4.10062955e-02,
       4.09839437e-02, 4.09618692e-02, 4.09400737e-02, 4.09185586e-02,
       4.08973257e-02, 4.08763764e-02, 4.08557122e-02, 4.08353348e-02,
       4.08152454e-02, 4.07954456e-02, 4.07759368e-02, 4.07567204e-02,
       4.07377977e-02, 4.07191701e-02, 4.07008389e-02, 4.06828054e-02,
       4.06650708e-02, 4.06476363e-02, 4.06305032e-02, 4.06136726e-02,
       4.05971457e-02, 4.05809236e-02, 4.05650073e-02, 4.05493980e-02,
       4.05340966e-02, 4.05191042e-02, 4.05044218e-02, 4.04900502e-02,
       4.04759904e-02, 4.04622434e-02, 4.04488098e-02, 4.04356907e-02,
      

In [33]:
import scipy.io as scio
from scipy.io import savemat

# Visualizing the results for different a_k(x)

In [34]:
def plot_models(models):
    
    xx       = np.linspace(0,2,1000).reshape(-1,1)
    fig, ax  = plt.subplots(1,1,figsize=(4,3),dpi=200,constrained_layout=True)
    
    for k,model_name in enumerate(models):
        model.load_state_dict(torch.load(model_name))
        U_NN     = model.get_U_np(xx)
        U_NN_min = U_NN.min()
        U_NN     = U_NN-U_NN_min
        # c        = ax.plot(xx[:,0],U_NN,'-',lw=1.5,label="$a_{%d}(x)$"%(k+1))
        c        = ax.plot(xx[:,0],U_NN,'-',lw=1.5,label="bongo"%(k+1))

    ax.legend(fontsize=10)
    # ax.set_xlabel('$x$',fontsize=10)
    # ax.set_ylabel('$U(x)$',fontsize=10)
    ax.set_xlim([0,2])
    ax.set_yticks([0,0.2,0.4,0.6,0.8,1,1.2])
    ax.set_xticks([0,.5,1.,1.5,2])
    ax.yaxis.grid(linestyle='--')
    ax.tick_params(axis="both", labelsize=10)
    plt.show()
    
plot_models(["savee/model_anaconda3"])
#plot_models(["savee/model_a1_update", "savee/model_a2_update", "savee/model_a3_update", "savee/model_a4", "savee/model_a5_update"])

/var/folders/jf/w5m4c3sj361fdh8b5wz1cy1m0000gn/T/ipykernel_78372/3117479736.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name))

FileNotFoundError: [Errno 2] No such file or directory: 'savee/model_anaconda3'

Error in callback <function _draw_all_if_interactive at 0x10fdac2c0> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 800x600 with 1 Axes>

In [ ]:
model

In [ ]:
torch.load("/Users/annacoletti/Desktop/savee/model_a3_update.mat")